# Setup

This notebook illustrates how to access the Many-Affine BBOB problems using IOHexperimenter and create a submission to the corresponding competition.
For more information about the MA-BBOB generator's construction, please see [this paper](https://arxiv.org/abs/2312.11083).

To install IOHexperimenter, the following command can be used:

In [1]:
#%pip install ioh

Please make sure that you use version >= 0.3.15 to follow the examples in this notebook.
Once installed, simply import the package

In [1]:
import ioh

# Accessing problems

In ioh, everything revolves around the problem class, so we start by creating an individual problem. For MA-BBOB, we have a generator which can create arbitrary problem instances. This generator can be called using either an instance number, or by providing information about the weights, instances and optima location of the components. We start of by illustrating the former approach:

In [4]:
f = ioh.problem.ManyAffine(instance = 1, n_variables = 5)

The resulting problem can be used as any other in iohexperimenter, for more details please see [the documentation](https://iohprofiler.github.io/IOHexperimenter/python.html)

One difference between the MA-BBOB problems and the regular BBOB problems in terms of their usage is in the function ID. 
By default, all problems created using the generator have a problem ID of 0. It might be useful for analysis to change this to a different number, which can be done using:

In [6]:
f.set_id(100)

To generate specific MA-BBOB problems, for example to have baseline data to compare to, you can utilize the alternative constructor as follows (note: the data used here is available on [Zenodo](https://doi.org/10.5281/zenodo.8208572)):

In [14]:
import pandas as pd
import numpy as np

weights = pd.read_csv("weights.csv", index_col=0)
iids = pd.read_csv("iids.csv", index_col=0)
opt_locs = pd.read_csv("opt_locs.csv", index_col=0)

idx = 0
dim = 5

f_new = ioh.problem.ManyAffine(xopt = np.array(opt_locs.iloc[idx])[:dim], 
                               weights = np.array(weights.iloc[idx]),
                               instances = np.array(iids.iloc[idx]), 
                               n_variables = dim)
f_new.set_id(idx)
f_new.set_instance(idx)

In [15]:
f_new

<RealSingleObjectiveProblem 0. ManyAffine (iid=0 dim=5)>

## Creating your algorithm

To submit to the MA-BBOB competition, you simply need to create an optimization algorithm which accepts an ioh-problem as its input to the 'call' function. 
Your submission has to include the code itself as a single python file, but any additional information (e.g. performance data on the given instances) can also be included in the repository. 
In addition to the code, you should include a readme file (markdown) which describes the algorithm, and any relevant information about its execution.

For evaluation, we will generate a new set of instances and run all submissions, as well as our baselines. Each algorithm has a budget of 10000*dimensionality, and will be run 10 times with different seeds. As such, it is important that your algorithm accepts a seed as input!

Here, we show an example structure for a RandomSearch algorithm which would be a valid entry to the competition:

In [31]:
class RandomSearch:
    'Simple random search algorithm'
    def __init__(self, budget_factor: int = 10000):
        self.budget_factor: int = budget_factor
        
    def __call__(self, problem: ioh.problem.RealSingleObjective, seed: int) -> None:
        'Evaluate the problem n times with a randomly generated solution'
        rng = np.random.RandomState(seed)
        for _ in range(self.budget_factor * problem.meta_data.n_variables):
            x = rng.uniform(problem.bounds.lb, problem.bounds.ub)
            problem(x)             

## Testing your algorithm

To benchmark your algorithm, you can use any MA-BBOB instances. Here, we show an example using the instances from [Zenodo](https://doi.org/10.5281/zenodo.8208572) 

In [27]:
import os

logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="MAdata",       # in a folder named: 'my-experiment'
    algorithm_name="RandomSearch",    # meta-data for the algorithm used to generate these results
    store_positions=False               # disable storing x-variables in the logged files
)

In [28]:
RS = RandomSearch()
for idx in range(5):

    f_new = ioh.problem.ManyAffine(xopt = np.array(opt_locs.iloc[idx])[:dim], 
                                   weights = np.array(weights.iloc[idx]),
                                   instances = np.array(iids.iloc[idx]), 
                                   n_variables = dim)
    f_new.set_id(idx)
    f_new.set_instance(idx)
    f_new.attach_logger(logger)
    
    for seed in range(10):
        RS(f_new, seed)
        f_new.reset()
logger.close()

You can then upload your data to IOHanalyzer as normal, or alternatively process the files directly to get the AOCC values to compare with the processed data from [Zenodo](https://doi.org/10.5281/zenodo.8208572) 

In [52]:
def get_auc_table(fname):
    max_budget = 10000
    dt = pd.read_csv(fname, sep=' ', decimal=',')
    dt = dt[dt['raw_y'] != 'raw_y'].astype(float)
    dt['run_id'] = np.cumsum(dt['evaluations'] == 1)
    items = []
    for run in np.unique(dt['run_id']):
        dt_temp = dt[dt['run_id'] == run]
        new_row = pd.DataFrame({'evaluations' : max_budget, 'raw_y':min(dt_temp['raw_y']), 'run_id' : run}, index=[0])
        dt_temp = pd.concat([dt_temp, new_row], ignore_index=True)
        auc = np.sum((2 - np.clip(np.log10(dt_temp['raw_y'][:-1]), -8, 2)) * np.ediff1d(dt_temp['evaluations']))/(max_budget*10) 
        items.append([fname, auc, run])
    dt_auc = pd.DataFrame.from_records(items, columns=['fname', 'aocc', 'run'])
    return dt_auc

In [53]:
aocc_idx0 = get_auc_table("MAdata/data_f0_ManyAffine/IOHprofiler_f0_DIM5.dat")

The competition gets judged on average AOCC value across all test instances, using the same bounds and scaling as the function above. 

# Questions

If you have any questions about using the MA-BBOB generator, or the MA-BBOB competition, please contact [Diederick Vermetten](mailto:d.l.vermetten@liacs.leidenuniv.nl)

In [56]:
import glob

import numpy as np
from shutil import rmtree

def clean():
    for name in ("MAdata"):
        for path in glob.glob(f"{name}*"):
            if os.path.isfile(path):
                os.remove(path)
            if os.path.isdir(path):
                rmtree(path, ignore_errors=True)

clean()